In [ ]:
# Source the package setup script
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/00_setup_packages.R")

# Source the graphing functions
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/01_graphing_functions.R", local = knitr:::knit_global())

# Model 1

---

## Question

How do environmental factors (**microhabitat**, **nudibranch presence**) affect P. cristatus occurrence across individuals, sexes, and morphs?

---

## Objective

Test for the effect of **microhabitat type** and **aeolid nudibranch presence** on the occurrence of **pods**. 

---

## Method

### 1. Load cleaned data.

We start by loading the cleaned data from the "02_data_cleaning" pipeline. This data has already undergone transformations and contains relevant metrics for our models.


In [ ]:
data_m1_clean <- read.csv("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m1_clean.csv")


---

### 2. Prepare data for modeling.

#### **Categorical predictors**

For categorical and binary predictors, R automatically dummy-codes the variables. For binary variables, such as Sex (0 = Female, 1 = Male), R sets the baseline to 0 unless specified otherwise. This ensures comparisons are made relative to the baseline group.

#### **Continuous predictors**

We standardize continuous predictors, such as Fecundity, by centering and scaling them (dividing by two standard deviations). This improves interpretability, aligning their coefficients with those of binary predictors, as suggested by Gelman (2008).


In [ ]:
# Convert categorical variables to factors
columns_to_convert_m1 <- c("Site", "Quadrat", "Microhabitat", "Pod", "Nudibranch")

data_m1_clean <- data_m1_clean %>%
    mutate(across(all_of(columns_to_convert_m1), as.factor))

# Set reference category for categorical predictors
data_m1_clean$Microhabitat <- relevel(data_m1_clean$Microhabitat, ref = "Red_Algae")
data_m1_clean$Nudibranch <- relevel(data_m1_clean$Nudibranch, ref = "0")


# Convert continuous variables to numerical
# NONE ARE CONTINUOUS


---

### 3. Visualize response variable distributions

To understand the distributions of our response variables, we plot density curves. This helps confirm whether a Gamma distribution is appropriate for modeling, as Gamma is suited for positively skewed, continuous data.


In [ ]:
# Function to generate stacked bar plots for morph occurrence
generate_pod_occurrence_plot <- function(
  data, 
  variable, 
  sample_size_data, 
  color, 
  axis_title_x = TRUE, 
  axis_text_x = TRUE, 
  axis_text_y = TRUE,
  axis_title_y = TRUE,
  plot_title = NULL,
  axis_title_size = 10, 
  axis_text_size = 8 
) {
  data %>%
    ggplot(aes(x = Microhabitat, fill = factor(!!sym(variable)))) +
    geom_bar(position = "fill") +
    scale_fill_manual(
      values = c("1" = color, "0" = "lightgrey"),
      labels = c("1" = "Present", "0" = "Absent")
    ) +
    labs(
      title = plot_title,
      x = if (isTRUE(axis_title_x)) "Microhabitat" else NULL,
      y = if (isTRUE(axis_title_y)) "Proportion of Quadrats" else NULL,
      fill = NULL
    ) +
    theme_bw(base_size = 10) +
    theme(
      axis.title.y = if (axis_title_y) element_text(size = axis_title_size) else element_blank(),
      axis.text.y  = if (axis_text_y) element_text(size = axis_text_size) else element_blank(),
      axis.title.x = if (axis_title_x) element_text(size = axis_title_size) else element_blank(),
      axis.text.x  = if (axis_text_x) element_text(size = axis_text_size, angle = 45, hjust = 1) else element_blank(),
      legend.position = "none",
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank()
    ) +
    geom_text(
      data = sample_size_data,
      aes(x = Microhabitat, y = 1.05, label = paste("n =", sample_size)),
      vjust = 0, size = 2, inherit.aes = FALSE
    )
}


# Prepare data for sample sizes
data_m1_sample_size <- data_m1_clean %>%
    group_by(Microhabitat) %>%
    summarise(sample_size = n(), .groups = "drop")

# Generate plot for pod occurrence
plot_m1_occurrence <- generate_pod_occurrence_plot(
    data = data_m1_clean,
    variable = "Pod",
    sample_size_data = data_m1_sample_size,
    color = "gray20",
    axis_title_x = FALSE,
    axis_text_x = TRUE,
    axis_title_y = TRUE,
    axis_title_size = 10,
    axis_text_size = 8
    )


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_m1_occurrence.png", plot = plot_m1_occurrence, width = 3, height = 3, units = "in", dpi = 300)


In [ ]:
# Convert images to base64
plot_m1_occurrence <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_m1_occurrence.png")

# Create the HTML 
html_occurrence <- paste0("
  <div style='text-align: center; margin: 20px auto; max-width: 600px;'>
    <img src='", plot_m1_occurrence, "' alt='Podocerus Occurrence Plot' style='width: 100%; height: auto; border: 1px solid #ccc;'/>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_occurrence)


---

### 4. Justification for GLMs and Bayesian methods

#### **Why GLMs?**

Generalized Linear Models (GLMs) are used because our response variable is binary. Linear models assume a continuous range of values for the response variable, which is not suitable for binary outcomes. GLMs address this limitation by introducing a link function that transforms the linear predictor into a form suitable for the response variable's distribution.

---

#### **Why Bayesian methods?**

Bayesian GLMs were chosen over frequentist approaches because:

-   **Sparse data**: Bayesian methods handle sparse datasets more robustly.

-   **Priors**: They allow us to incorporate prior knowledge, improving model performance.

-   **Posterior distributions**: Bayesian models provide posterior distributions, offering a full view of parameter uncertainty.

---

### 5. Define priors

#### **What are priors?**

In Bayesian statistics, priors represent our beliefs about parameter values before analyzing the data. These beliefs are mathematically expressed as probability distributions. Priors guide the model, especially when data are sparse or when the signal-to-noise ratio is low.

---

#### **Why priors matter:**

-   **Prevent overfitting**: Priors discourage extreme parameter estimates unless strongly supported by the data.

-   **Balance restrictiveness and flexibility**: Weakly informative priors let the data dominate while providing reasonable bounds.

-   **Leverage existing knowledge**: Informative priors incorporate previous research or domain expertise, improving accuracy in well-studied systems.

---

#### **How priors work in this analysis:**
We combine the priors (representing initial beliefs) with the likelihood of the observed data to compute posterior distributions, which reflect updated beliefs after observing the data.

The general formula is:

$$
\begin{aligned}
\text{Posterior } \alpha \text{ Likelihood} * \text{Prior}
\end{aligned}
$$

---

#### **Model family and formula**

The response variable (Pod presence) is binary (1 = present, 0 = absent). We model this variable using a Bernoulli distribution with a logit link function:
$$
y_{i} \sim \text{Bernoulli}(p_{i}) \\
$$

The probability of occurrence $p_i$ is modeled as:

$$
\text{log-Odds} = \text{logit}(p_{i}) = \beta_{0} + \beta x_{i} \\
$$

Where:

-   **$y_{i}$**: Response variable for observation $i$

-   **$p_{i}$**: Probability of *P. cristatus* presence for $i$ (on the log scale)

-   **$\beta_{0}$**: Intercept, mean value of $\text{logit}(p)$ (i.e., log-Odds) when predictors are at baseline

-   **$\beta$**: Coefficient/slope for predictor $x_{i}$, representing the effect of $x_{i}$ on $\text{logit}(p_i)$ (i.e., log-Odds).

---

#### Chosen priors and rationale

**Intercept (\beta_{0})**: The intercept (\beta_{0}) reflects the baseline log-odds of *P. cristatus* occurrence in the reference conditions (Red Algae with no nudibranchs). Preliminary observations from SCUBA suggest a baseline probability of approximately 50% for P. cristatus presence, translating to:

$$
\begin{aligned}
\text{logit}(0.50) = log(\frac{0.50}{1-0.50}) = 0
\end{aligned}
$$

We set a weakly informative prior:
$$
\begin{aligned}
\beta_{0} \approx N(0,1)
\end{aligned}
$$

This prior allows for moderate variability in log-odds (95% of values between -2 and 2), corresponding to probabilities ranging from 12% to 88% on the original scale:

$$
\begin{aligned}
p = \frac{e^{\text{logit}(p)}}{1 + e^{\text{logit}(p)}}
\end{aligned}
$$

For $\text{logit}(p)=0$:
$$
\begin{aligned}
p = \frac{e^{0}}{1 + e^{0}} = \frac{1}{1+1} = 0.50
\end{aligned}
$$

For $\text{logit}(p)=-2$:
$$
\begin{aligned}
p = \frac{e^{-2}}{1 + e^{-2}} = \frac{0.135}{1+0.135} = 0.12
\end{aligned}
$$

For $\text{logit}(p)=2$:
$$
\begin{aligned}
p = \frac{e^{2}}{1 + e^{2}} = \frac{7.39}{1+7.39} = 0.88
\end{aligned}
$$

---

**Slope ($\beta$)**: The slope represents the change in log-odds of occurrence for a one-unit increase in the predictor. On the odds scale, the slope is interpreted as a multiplicative scaling factor:

$$
\begin{aligned}
\text{Odds}_{x+1} = \text{Odds}_{x} * e^{\beta}
\end{aligned}
$$

For example:

-   If $\beta = 0.1$, a one-unit increase in the predictor scales the Odds response by $e^{0.1} \approx 1.10$ (a 10% increase).

-   If $\beta = -0.1$, a one-unit increase scales the Odds response by $e^{-0.1} \approx 0.91$ (a 9% decrease).

We don't know how our predictors will affect the response, so we use a weakly informative prior:

$$
\begin{aligned}
\beta \approx N(0,0.5)
\end{aligned}
$$

This prior assumes no effect of the predictor on average ($e^{0} = 1$, and allows moderate positive or negative effects, spanning approximately $[-1, 1]$ on the log scale ($e^{-1} \approx 0.37$ to $e^{1} \approx 2.72$ on the original scale). So the Odds of occurrence will not exceed a minimum of $\text{Odds} * 0.37$ and maximum of $\text{Odds} * 2.72$.

---

**Random effects ($\sigma$)**: Random effects account for variability between groups that we are not explicitly testing. In this model, sampling site will be the random effect. This is because we have repeated measures within the same Sites. Measurements within the same Site are likely to be correlated due to shared environmental factors, spatial proximity, or other unobserved factors specific to the site. Including random effects in our model accounts for this. To capture moderate variability, we assign:

$$
\begin{aligned}
\approx N(0,0.5)
\end{aligned}
$$

This ensures flexibility without overfitting.



---

#### **Visualizing priors**

To validate these priors, we run models sampling only from the priors (sample_prior = "only") and inspect their outputs to ensure they align with our expectations.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# m1_priors <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Pod ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m1_clean,
#     prior = c(
#         prior(normal(0, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = "only",
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m1_priors, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1_priors.rds")

In [ ]:
# Load models
m1_priors <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1_priors.rds")

# Extract prior model results.
prior_samples_m1 <- as_draws_df(m1_priors)
prior_samples_m1 <- as.data.frame(prior_samples_m1)

In [ ]:
# List of datasets and labels
prior_samples <- list(
    Pod = prior_samples_m1
)

#Custom labels
custom_labels_priors <- c(
  "b_MicrohabitatOther_Hydroids" = "Other Hydroids",
  "b_MicrohabitatSertulariidae" = "Sertulariidae",
  "b_MicrohabitatHydrallmania" = "Hydrallmania",
  "b_MicrohabitatLafoea" = "Lafoea",
  "b_MicrohabitatSertulariidae_Bryozoa" = "Sertulariidae with Bryozoa",
  "b_Nudibranch1" = "Nudibranch"
)

custom_labels_priors_intercept <- c(
  "b_Intercept" = "Intercept"
)


# Generate plots for predictors
prior_plots <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
                "b_MicrohabitatOther_Hydroids",
                "b_MicrohabitatSertulariidae",
                "b_MicrohabitatHydrallmania",
                "b_MicrohabitatLafoea",
                "b_MicrohabitatSertulariidae_Bryozoa",
                "b_Nudibranch1"
            ),
            x_range = c(-5, 5),
            custom_labels = custom_labels_priors,
            axis_title_y = label %in% c("Pod")
        )
    }
)
names(prior_plots) <- names(prior_samples)


# Generate plot for intercept
prior_plots_intercept <- lapply(
    names(prior_samples),
    function(label) {
        generate_posterior_plot(
            prior_samples[[label]],
            regex_pars = c(
                "b_Intercept"
            ),
            x_range = c(-10, 10),
            custom_labels = custom_labels_priors_intercept,
            axis_title_y = label %in% c("Pod")
        )
    }
)
names(prior_plots_intercept) <- names(prior_samples)



# Add grey bars with labels for each plot
prior_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    prior_plots,
    names(prior_plots),
    SIMPLIFY = FALSE
)

prior_plots_intercept_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar(label)) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    prior_plots_intercept,
    names(prior_plots_intercept),
    SIMPLIFY = FALSE
)


# Combine plots into a 2x3 grid (we don't really need to do this here because there is only 1 response variable). But we are being consistent.
plot_priors_m1 <- patchwork::wrap_plots(
    prior_plots_with_bars[c("Pod")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")

plot_priors_intercept_m1 <- patchwork::wrap_plots(
    prior_plots_intercept_with_bars[c("Pod")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")


# Add a unified x-axis label
plot_priors_m1 <- plot_priors_m1 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

plot_priors_intercept_m1 <- plot_priors_intercept_m1 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )



ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m1.png", plot = plot_priors_m1, width = 4.5, height = 4.5, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m1.png", plot = plot_priors_intercept_m1, width = 4.5, height = 4.5, units = "in", dpi = 300)


In [ ]:
# Convert images to base64 (assuming these return base64 data URIs)
plot_priors_m1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_m1.png")
plot_priors_intercept_m1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_priors_intercept_m1.png")

# Create the HTML 
html_priors <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_priors_m1, "' alt='Prior Plot'>
    <img src='", plot_priors_intercept_m1, "' alt='Prior Intercept Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_priors)


---

### 6. Run final models

Now that we have finalized the model parameters, we fit models using the actual data and compare them to null models to assess the significance of predictors.


>**Note:** We will run these models in RStudio to be consistent because the rstan package sometimes does not like Jupyter. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Set seed for reproducibility
# set.seed(5678)
# # null model (for comparison)
# m1v0 <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Pod ~ 1 + (1 | Site),
#     data = data_m1_clean,
#     prior = c(
#         prior(normal(0, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m1v0, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v0.rds")

# # Set seed for reproducibility
# set.seed(5678)
# # Model with no interaction effects
# m1v1 <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Pod ~ 1 + Microhabitat + Nudibranch + (1 | Site),
#     data = data_m1_clean,
#     prior = c(
#         prior(normal(0, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m1v1, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v1.rds")

# # Set seed for reproducibility
# set.seed(5678)
# # Model with interaction effects
# m1v2 <- brm(
#     family = bernoulli(link = "logit"),
#     formula = Pod ~ 1 + Microhabitat * Nudibranch + (1 | Site),
#     data = data_m1_clean,
#     prior = c(
#         prior(normal(0, 1), class = "Intercept"),
#         prior(normal(0, 0.5), class = "b"),
#         prior(normal(0, 0.5), class = "sd")
#     ),
#     sample_prior = TRUE,
#     save_pars = save_pars(all = TRUE),
#     control = list(adapt_delta = 0.99, max_treedepth = 12),
#     iter = 15000,
#     warmup = 5000,
#     chains = 2,
#     cores = parallel::detectCores(logical=FALSE),
#     backend = "rstan"
# )
# saveRDS(m1v2, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v2.rds")


#### **Model comparison**

Make sure final model outperforms the null model. Use LOO to compare models.


>**Note:** We will added loo to these models in RStudio. The models were saved in RStudio and loaded below. We left the code chunks as comments for reference.

In [ ]:

# # Add LOO to models
# m1v0 <- add_criterion(m1v0, "loo", moment_match = TRUE)
# m1v1 <- add_criterion(m1v1, "loo", moment_match = TRUE)
# m1v2 <- add_criterion(m1v2, "loo", moment_match = TRUE)

# Save models with loo so you don't have to do this again
# saveRDS(m1v0, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v0.rds")
# saveRDS(m1v1, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v1.rds")
# saveRDS(m1v2, file = "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v2.rds")


In [ ]:
# Load models
m1v0 <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v0.rds")
m1v1 <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v1.rds")
m1v2 <- readRDS("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/models/m1v2.rds")


In [ ]:

# Compare models
loo1 <- loo_compare(m1v0, m1v1, m1v2)

In [ ]:
# Convert to dataframe
df_loo1 <- as.data.frame(loo1) %>%
  rownames_to_column(var = "Model")

  # Save loo as tables
write.table(df_loo1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_loo_m1.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:
# Convert each data frame to a plain HTML table string
table_1_loo <- minimal_html_table(df_loo1, caption = "LOO values - Probability of Pod Occurrence")

my_tabs_loo <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_loo:checked ~ #content1_loo {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_loo:checked + label[for="tab1_loo"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_loo" id="tab1_loo" checked>
  <label class="tab-label" for="tab1_loo">Table 1</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_loo">REPLACE_WITH_table_1</div>
</div>
'

# Now do the replacements for each table
my_tabs_loo <- gsub("REPLACE_WITH_table_1", table_1_loo, my_tabs_loo)

IRdisplay::display_html(my_tabs_loo)


#### **Visualize posteriors**

We extract the final model results and visualize the posteriors of our model parameters to get an idea of the significance of the results. This is what we did above when we were evaluating our prior.

In [ ]:

# Extract results
posterior_samples_m1v1 <- as_draws_df(m1v1)
posterior_samples_m1v2 <- as_draws_df(m1v2)


In [ ]:

# List of datasets and labels
predictor_samples <- list(
    Pod = posterior_samples_m1v1
)

predictor_samples_interaction <- list(
    Pod = posterior_samples_m1v2
)

# Baseline category data
baseline_data <- tibble(
  parameter = "Red_Algae",
  mean = 0,  # Centered at 0
  ci_low = -0.2,  # Example CI range
  ci_high = 0.2
)

# Order categories so "Red Algae" appears at the bottom
parameter_order <- c(
  "Red_Algae",
  "b_MicrohabitatOther_Hydroids",
  "b_MicrohabitatSertulariidae",
  "b_MicrohabitatHydrallmania",
  "b_MicrohabitatLafoea",
  "b_MicrohabitatSertulariidae_Bryozoa",
  "b_Nudibranch1"
)

#Custom labels
custom_labels_posteriors <- c(
  "Red_Algae" = "Red Algae",
  "b_MicrohabitatOther_Hydroids" = "Other Hydroids",
  "b_MicrohabitatSertulariidae" = "Sertulariidae",
  "b_MicrohabitatHydrallmania" = "Hydrallmania",
  "b_MicrohabitatLafoea" = "Lafoea",
  "b_MicrohabitatSertulariidae_Bryozoa" = "Sertulariidae with Bryozoa",
  "b_Nudibranch1" = "Aeolid Nudibranch Presence"
)


# Convert the y-axis parameters to factors for consistent alignment
baseline_data$parameter <- factor(baseline_data$parameter, levels = parameter_order)



# Generate plots for predictors
predictor_plots <- lapply(
    names(predictor_samples),
    function(label) {
        generate_posterior_plot(
            predictor_samples[[label]],
            regex_pars = c(
                "b_MicrohabitatOther_Hydroids",
                "b_MicrohabitatSertulariidae",
                "b_MicrohabitatHydrallmania",
                "b_MicrohabitatLafoea",
                "b_MicrohabitatSertulariidae_Bryozoa",
                "b_Nudibranch1"
            ),
            x_range = c(-2, 2),
            custom_labels = custom_labels_posteriors,
            axis_title_y = label %in% c("Pod")
        ) +
        geom_point(
            data = baseline_data,
            aes(x = mean, y = parameter),
            inherit.aes = FALSE,
            color = "dodgerblue4",
            size = 2
        )
    }
)
names(predictor_plots) <- names(predictor_samples)


predictor_plots_interaction <- lapply(
    names(predictor_samples_interaction),
    function(label) {
        generate_posterior_plot(
            predictor_samples_interaction[[label]],
            regex_pars = c(
                "^b_MicrohabitatOther_Hydroids$",
                "^b_MicrohabitatSertulariidae$",
                "^b_MicrohabitatHydrallmania$",
                "^b_MicrohabitatLafoea$",
                "^b_MicrohabitatSertulariidae_Bryozoa$",
                "^b_Nudibranch1$"
            ),
            x_range = c(-2, 2),
            custom_labels = custom_labels_posteriors,
            axis_title_y = label %in% c("Pod")
        ) +
        geom_point(
          data = baseline_data,
          aes(x = mean, y = parameter),
          inherit.aes = FALSE,
          color = "dodgerblue4",
          size = 2
        )
    }
)
names(predictor_plots_interaction) <- names(predictor_samples_interaction)




# Add grey bars with labels for each plot

predictor_plots_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar("Pod Occurrence")) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    predictor_plots,
    names(predictor_plots),
    SIMPLIFY = FALSE
)

predictor_plots_interaction_with_bars <- mapply(
    function(plot, label) {
        patchwork::wrap_elements(create_top_bar("Pod Occurrence\n(with Interaction Effects)")) / plot +
            patchwork::plot_layout(heights = c(0.2, 1)) # Adjust heights to reduce spacing
    },
    predictor_plots_interaction,
    names(predictor_plots_interaction),
    SIMPLIFY = FALSE
)



# Combine plots into a 2x3 grid
plot_posteriors_m1v1 <- patchwork::wrap_plots(
    predictor_plots_with_bars[c("Pod")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")

plot_posteriors_m1v2 <- patchwork::wrap_plots(
    predictor_plots_interaction_with_bars[c("Pod")],
    ncol = 1
) +
    patchwork::plot_layout(guides = "collect")


# Add a unified x-axis label
plot_posteriors_m1v1 <- plot_posteriors_m1v1 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

plot_posteriors_m1v2 <- plot_posteriors_m1v2 +
    patchwork::plot_annotation(
        caption = "Expected value of the odds response (logit scale)",
        theme = theme(plot.caption = element_text(hjust = 0.65, size = 10))
    )

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m1v1.png", plot = plot_posteriors_m1v1, width = 4.5, height = 4.5, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m1v2.png", plot = plot_posteriors_m1v2, width = 4.5, height = 4.5, units = "in", dpi = 300)

In [ ]:
# Convert images to base64
plot_posteriors_m1v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m1v1.png")

plot_posteriors_m1v2 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/plot_posteriors_m1v2.png")

# Create the HTML 
html_posteriors <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", plot_posteriors_m1v1, "' alt='Posterior Plot'>
    <img src='", plot_posteriors_m1v2, "' alt='Interaction Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors)


The graph of the posteriors gives us an idea of the significance of each predictor. We need to follow up with an evaluation of the model performance before we can trust these results.

---

### 7. Evaluate model performance

#### **Trace plots**

Visualize parameter sampling across iterations to confirm convergence. Each chain should wander around the same mean value without any strong upward or downward trends. "fuzzy caterpillar" or "horizontal band." 


In [ ]:
# Generate trace plots for all models
trace_plots_m1v1 <- generate_trace_plot(
  model = m1v1,
  regex_pars = c("b_MicrohabitatOther_Hydroids", "b_MicrohabitatSertulariidae", "b_MicrohabitatHydrallmania", "b_MicrohabitatLafoea", "b_MicrohabitatSertulariidae_Bryozoa", "b_Nudibranch1"),
  plot_title = "Pod Occurrence Model",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)

trace_plots_m1v2 <- generate_trace_plot(
  model = m1v2,
  regex_pars = c("^b_MicrohabitatOther_Hydroids$", "^b_MicrohabitatSertulariidae$", "^b_MicrohabitatHydrallmania$", "^b_MicrohabitatLafoea$", "^b_MicrohabitatSertulariidae_Bryozoa$", "^b_Nudibranch1$"),
  plot_title = "Pod Occurrence Model (with Interaction Effects)",
  axis_title_y = TRUE,
  axis_text_x = TRUE,
  axis_title_size = 10,
  axis_text_size = 8
)


ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m1v1.png", plot = trace_plots_m1v1, width = 6, height = 3, units = "in", dpi = 300)
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m1v2.png", plot = trace_plots_m1v2, width = 6, height = 3, units = "in", dpi = 300)

In [ ]:
# Convert images to base64 (if not already done)
trace_plots_m1v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m1v1.png")
trace_plots_m1v2     <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/trace_plots_m1v2.png")


# Create the HTML 
html_trace_plots <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(1, 1fr); /* 1 columns per row */
    gap: 3px;
    padding: 3px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 600px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", trace_plots_m1v1, "' alt='Trace Plot'>
  <img src='", trace_plots_m1v2, "' alt='Trace Plot (with interactions)'>
</div>
")


# Display the HTML
IRdisplay::display_html(html_trace_plots)


#### **Posterior predictive checks**

Simulate data based on the models and compare to observed data to verify goodness-of-fit. We do this using "pp_check". The observed data (black line/dots) should sit comfortably within the distribution of simulated data (colored areas or lines)

In [ ]:

# Generate posterior predictive check plots for all models
pp_check_plots_m1v1 <-  generate_pp_check(
  model = m1v1,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Pod Occurrence Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)

pp_check_plots_m1v2 <-  generate_pp_check(
  model = m1v2,
  nreps = 100,
  axis_title_y = TRUE,
  y_label = "Density",
  plot_title = "Pod Occurrence (with Interactions) Model",
  axis_title_size = 10,  # custom title size
  axis_text_size = 8    # custom tick label size
)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m1v1.png", plot = pp_check_plots_m1v1, width = 3, height = 3, units = "in", dpi = 300)

ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m1v2.png", plot = pp_check_plots_m1v2, width = 3, height = 3, units = "in", dpi = 300)

In [ ]:
# Convert images to base64
pp_check_plots_m1v1 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m1v1.png")
pp_check_plots_m1v2     <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/pp_check_plots_m1v2.png")


# Create the HTML (horizontal display)
html_pp_check_plots <- paste0("
<style>
  .grid-container {
    display: grid;
    grid-template-columns: repeat(2, 1fr); /* 2 columns per row */
    gap: 15px;
    padding: 10px;
    justify-items: center;
  }

  .grid-container img {
    max-width: 300px;
    width: 100%;
    height: auto;
    border: 1px solid #ccc;
  }
</style>

<div class='grid-container'>
  <img src='", pp_check_plots_m1v1, "' alt='Plot'>
  <img src='", pp_check_plots_m1v2, "' alt='Plot with interactions'>
</div>
")

IRdisplay::display_html(html_pp_check_plots)



#### **Check convergence**

Check that all $\hat{R}$ values are close to 1, indicating good convergence.

In [ ]:
# Create a helper function
extract_rhat <- function(model, model_name) {
  rhat(model) %>%
    as.data.frame() %>%
    rownames_to_column(var = "Parameter") %>%
    dplyr::filter(startsWith(Parameter, "b_")) %>%   # <-- keep only b_ terms
    dplyr::rename(Rhat = 2) %>%
    dplyr::mutate(Model = model_name) %>%
    dplyr::mutate(across(where(is.numeric), ~ signif(.x, digits = 3)))
}

# Extract for each model group
rhat1 <- extract_rhat(m1v1,                "Pod")
rhat2 <- extract_rhat(m1v2,    "Pod")

In [ ]:
# Save tables
write.table(rhat1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m1v1.csv", sep = ",", row.names = FALSE, col.names = TRUE)
write.table(rhat2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_rhat_m1v2.csv", sep = ",", row.names = FALSE, col.names = TRUE)


In [ ]:
# Convert each data frame to a plain HTML table string
table_1_rhat <- minimal_html_table(rhat1, caption = "Rhat values - Probability of Pod Occurrence")
table_2_rhat <- minimal_html_table(rhat2, caption = "Rhat values - Probability of Pod Occurrence (with interaction effects)")


my_tabs_rhat <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_rhat:checked ~ #content1_rhat,
#tab2_rhat:checked ~ #content2_rhat {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_rhat:checked + label[for="tab1_rhat"],
#tab2_rhat:checked + label[for="tab2_rhat"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab1_rhat" checked>
  <label class="tab-label" for="tab1_rhat">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_rhat" id="tab2_rhat">
  <label class="tab-label" for="tab2_rhat">Table 2</label>


  <!-- Content for each tab -->
  <div class="tab-content" id="content1_rhat">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_rhat">REPLACE_WITH_table_2</div>
</div>
'

# Now do the replacements for each table
my_tabs_rhat <- gsub("REPLACE_WITH_table_1", table_1_rhat, my_tabs_rhat)
my_tabs_rhat <- gsub("REPLACE_WITH_table_2", table_2_rhat, my_tabs_rhat)


IRdisplay::display_html(my_tabs_rhat)


#### **Check uncertainty**

Extract parameter estimates and their confidence intervals to assess the significance of the predictors on color pattern metrics. We check 85% and 95% confidence intervals. Summaries are displayed in tables for all models. 

In [ ]:
# Extract summaries for each variable
extract_summary <- function(model, prob_85, prob_95) {
    summary_85 <- summary(model, prob = prob_85)
    summary_95 <- summary(model, prob = prob_95)

    as.data.frame(summary_85$fixed) %>%
        dplyr::select("Estimate", "Est.Error", "l-85% CI", "u-85% CI") %>%
        mutate(
            "l-95% CI" = summary_95$fixed$`l-95% CI`,
            "u-95% CI" = summary_95$fixed$`u-95% CI`
        ) %>%
        mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
        rownames_to_column(var = "Parameter") # Add rownames as Parameter column
}

uncertainty1 <- extract_summary(m1v1, prob_85 = 0.85, prob_95 = 0.95)
uncertainty2 <- extract_summary(m1v2, prob_85 = 0.85, prob_95 = 0.95)



In [ ]:
# Save tables
write.table(uncertainty1, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m1v1.csv", sep = ",", row.names = TRUE, col.names = TRUE)

write.table(uncertainty2, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_uncertainty_m1v2.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:

# Convert each data frame to a plain HTML table string
table_1_uncertainty <- minimal_html_table(uncertainty1, caption = "Uncertainty values - Probability of Pod Occurrence")
table_2_uncertainty <- minimal_html_table(uncertainty2, caption = "Uncertainty values - Probability of Pod Occurrence (with interaction effects)")


my_tabs_uncertainty <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1_uncertainty:checked ~ #content1_uncertainty,
#tab2_uncertainty:checked ~ #content2_uncertainty {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1_uncertainty:checked + label[for="tab1_uncertainty"],
#tab2_uncertainty:checked + label[for="tab2_uncertainty"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab1_uncertainty" checked>
  <label class="tab-label" for="tab1_uncertainty">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs_uncertainty" id="tab2_uncertainty">
  <label class="tab-label" for="tab2_uncertainty">Table 2</label>

  <!-- Content for each tab -->
  <div class="tab-content" id="content1_uncertainty">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2_uncertainty">REPLACE_WITH_table_2</div>
</div>
'

# Now do the replacements for each table
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_1", table_1_uncertainty, my_tabs_uncertainty)
my_tabs_uncertainty <- gsub("REPLACE_WITH_table_2", table_2_uncertainty, my_tabs_uncertainty)

IRdisplay::display_html(my_tabs_uncertainty)

---

#### **Check posterior probabilities**

Now we can evaluate our hypotheses using the posterior distributions of the model parameters. Remember, we are concerned with two things: 
1. Is there a difference in the probability of finding a pod between microhabitats? (Microhabitat effect)
2. Is there a difference in the probability of finding a pod between quadrats with and without aeolid nudibranchs? (Nudibranch effect)

We will calculate the posterior probability of these effects.

In [ ]:
##### **Posterior Probability of Microhabitat Effect**

draws <- as_draws_df(m1v1)

# Posterior probability that Other Hydroids, Sertulariidae, Hydrallmania, Lafoea, or Sertulariidae with Bryozoa are **more** likely to have Pods than Red Algae (reference microhabitat)
pp_Other_Hydroids_positive <- mean(draws$b_MicrohabitatOther_Hydroids > 0)
pp_Sertulariidae_positive <- mean(draws$b_MicrohabitatSertulariidae > 0)
pp_Hydrallmania_positive <- mean(draws$b_MicrohabitatHydrallmania > 0)
pp_Lafoea_positive <- mean(draws$b_MicrohabitatLafoea > 0)
pp_Sertulariidae_Bryozoa_positive <- mean(draws$b_MicrohabitatSertulariidae_Bryozoa > 0)

# Posterior probability that Other Hydroids, Sertulariidae, Hydrallmania, Lafoea, or Sertulariidae with Bryozoa are **less** likely to have Pods than Red Algae (reference microhabitat)
pp_Other_Hydroids_negative <- mean(draws$b_MicrohabitatOther_Hydroids < 0)
pp_Sertulariidae_negative <- mean(draws$b_MicrohabitatSertulariidae < 0)
pp_Hydrallmania_negative <- mean(draws$b_MicrohabitatHydrallmania < 0)
pp_Lafoea_negative <- mean(draws$b_MicrohabitatLafoea < 0)
pp_Sertulariidae_Bryozoa_negative <- mean(draws$b_MicrohabitatSertulariidae_Bryozoa < 0)


# Create a summary table
pp_summary_m1v1_microhabitat <- data.frame(
  Hypothesis = c(
    "P(Other Hydroids effect > 0)",
    "P(Sertulariidae effect > 0)",
    "P(Hydrallmania effect > 0)",
    "P(Lafoea effect > 0)",
    "P(Sertulariidae with Bryozoa effect > 0)",
    "P(Other Hydroids effect < 0)",
    "P(Sertulariidae effect < 0)",
    "P(Hydrallmania effect < 0)",
    "P(Lafoea effect < 0)",
    "P(Sertulariidae with Bryozoa effect < 0)"
  ),
  PosteriorProbability = c(
    pp_Other_Hydroids_positive,
    pp_Sertulariidae_positive,
    pp_Hydrallmania_positive,
    pp_Lafoea_positive,
    pp_Sertulariidae_Bryozoa_positive,
    pp_Other_Hydroids_negative,
    pp_Sertulariidae_negative,
    pp_Hydrallmania_negative,
    pp_Lafoea_negative,
    pp_Sertulariidae_Bryozoa_negative
  )
)

# Save table
write.table(pp_summary_m1v1_microhabitat, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m1v1_microhabitat.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m1v1_microhabitat, caption = "Posterior probabilities for Microhabitat Effect")

# Display it as HTML
IRdisplay::display_html(html_table)

In [ ]:
##### **Posterior Probability of Nudibranch Effect**

draws <- as_draws_df(m1v1)

# Posterior probability that quadrats with Aeolid Nudibranchs are **more** likely to have Pods than quadrats without Aeolid Nudibranchs
pp_nudie_positive <- mean(draws$b_Nudibranch1 > 0)

# Posterior probability that quadrats with Aeolid Nudibranchs are **less** likely to have Pods than quadrats without Aeolid Nudibranchs
pp_nudie_negative <- mean(draws$b_Nudibranch1 < 0)


# Create a summary table
pp_summary_m1v1_nudibranch <- data.frame(
  Hypothesis = c(
    "P(Nudibranch effect > 0)",
    "P(Nudibranch effect < 0)"
  ),
  PosteriorProbability = c(
    pp_nudie_positive,
    pp_nudie_negative
  )
)

# Save table
write.table(pp_summary_m1v1_nudibranch, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m1v1_nudibranch.csv", sep = ",", row.names = TRUE, col.names = TRUE)


In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m1v1_nudibranch, caption = "Posterior probabilities for Nudibranch Effect")

# Display it as HTML
IRdisplay::display_html(html_table)

In [ ]:
##### **Posterior Probability of Microhabitat Effect (with interactions)**

draws <- as_draws_df(m1v2)

# Posterior probability that Other Hydroids have a **higher** value than Red Algae (reference microhabitat)
pp_Other_Hydroids_positive <- mean(draws[["b_MicrohabitatOther_Hydroids"]] > 0)
pp_Sertulariidae_positive <- mean(draws[["b_MicrohabitatSertulariidae"]] > 0)
pp_Hydrallmania_positive <- mean(draws[["b_MicrohabitatHydrallmania"]] > 0)
pp_Lafoea_positive <- mean(draws[["b_MicrohabitatLafoea"]] > 0)
pp_Sertulariidae_Bryozoa_positive <- mean(draws[["b_MicrohabitatSertulariidae_Bryozoa"]] > 0)

# Posterior probability that females have a **lower** value than males
pp_Other_Hydroids_negative <- mean(draws[["b_MicrohabitatOther_Hydroids"]] < 0)
pp_Sertulariidae_negative <- mean(draws[["b_MicrohabitatSertulariidae"]] < 0)
pp_Hydrallmania_negative <- mean(draws[["b_MicrohabitatHydrallmania"]] < 0)
pp_Lafoea_negative <- mean(draws[["b_MicrohabitatLafoea"]] < 0)
pp_Sertulariidae_Bryozoa_negative <- mean(draws[["b_MicrohabitatSertulariidae_Bryozoa"]] < 0)


# Create a summary table
pp_summary_m1v2_microhabitat <- data.frame(
  Hypothesis = c(
    "P(Other Hydroids effect > 0)",
    "P(Sertulariidae effect > 0)",
    "P(Hydrallmania effect > 0)",
    "P(Lafoea effect > 0)",
    "P(Sertulariidae with Bryozoa effect > 0)",
    "P(Other Hydroids effect < 0)",
    "P(Sertulariidae effect < 0)",
    "P(Hydrallmania effect < 0)",
    "P(Lafoea effect < 0)",
    "P(Sertulariidae with Bryozoa effect < 0)"
  ),
  PosteriorProbability = c(
    pp_Other_Hydroids_positive,
    pp_Sertulariidae_positive,
    pp_Hydrallmania_positive,
    pp_Lafoea_positive,
    pp_Sertulariidae_Bryozoa_positive,
    pp_Other_Hydroids_negative,
    pp_Sertulariidae_negative,
    pp_Hydrallmania_negative,
    pp_Lafoea_negative,
    pp_Sertulariidae_Bryozoa_negative
  )
)

# Save table
write.table(pp_summary_m1v2_microhabitat, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m1v2_microhabitat.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m1v2_microhabitat, caption = "Posterior probabilities for Microhabitat Effect (with Nudibranch Interaction Effects)")

# Display it as HTML
IRdisplay::display_html(html_table)

In [ ]:
##### **Posterior Probability of Nudibranch Effect (with interactions)**

draws <- as_draws_df(m1v2)

# Posterior probability that quadrats with Aeolid Nudibranchs are **more** likely to have Pods than quadrats without Aeolid Nudibranchs
pp_nudie_positive <- mean(draws$b_Nudibranch1 > 0)

# Posterior probability that quadrats with Aeolid Nudibranchs are **less** likely to have Pods than quadrats without Aeolid Nudibranchs
pp_nudie_negative <- mean(draws$b_Nudibranch1 < 0)


# Create a summary table
pp_summary_m1v2_nudibranch <- data.frame(
  Hypothesis = c(
    "P(Nudibranch effect > 0)",
    "P(Nudibranch effect < 0)"
  ),
  PosteriorProbability = c(
    pp_nudie_positive,
    pp_nudie_negative
  )
)

# Save table
write.table(pp_summary_m1v2_nudibranch, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m1v2_nudibranch.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m1v2_nudibranch, caption = "Posterior probabilities for Nudibranch Effect (with Microhabitat Interaction Effects)")

# Display it as HTML
IRdisplay::display_html(html_table)

In [ ]:
##### **Posterior Probability of Microhabitat*Nudibranch Effect

draws <- as_draws_df(m1v2)

# Posterior probability that Pod occurrence on microhabitat with nudibranchs is > Red Algae (reference microhabitat)
pp_Other_Hydroids_nudie_positive <- mean(draws[["b_MicrohabitatOther_Hydroids:Nudibranch1"]] > 0)
pp_Sertulariidae_nudie_positive <- mean(draws[["b_MicrohabitatSertulariidae:Nudibranch1"]] > 0)
pp_Hydrallmania_nudie_positive <- mean(draws[["b_MicrohabitatHydrallmania:Nudibranch1"]] > 0)
pp_Lafoea_nudie_positive <- mean(draws[["b_MicrohabitatLafoea:Nudibranch1"]] > 0)
pp_Sertulariidae_Bryozoa_nudie_positive <- mean(draws[["b_MicrohabitatSertulariidae_Bryozoa:Nudibranch1"]] > 0)

# Posterior probability that Pod occurrence on microhabitat with nudibranchs is < Red Algae (reference microhabitat)
pp_Other_Hydroids_nudie_negative <- mean(draws[["b_MicrohabitatOther_Hydroids:Nudibranch1"]] < 0)
pp_Sertulariidae_nudie_negative <- mean(draws[["b_MicrohabitatSertulariidae:Nudibranch1"]] < 0)
pp_Hydrallmania_nudie_negative <- mean(draws[["b_MicrohabitatHydrallmania:Nudibranch1"]] < 0)
pp_Lafoea_nudie_negative <- mean(draws[["b_MicrohabitatLafoea:Nudibranch1"]] < 0)
pp_Sertulariidae_Bryozoa_nudie_negative <- mean(draws[["b_MicrohabitatSertulariidae_Bryozoa:Nudibranch1"]] < 0)


# Create a summary table
pp_summary_m1v2_microhabitat_nudibranch <- data.frame(
  Hypothesis = c(
    "P(Other Hydroids:Nudibranch effect > 0)",
    "P(Sertulariidae:Nudibranch effect > 0)",
    "P(Hydrallmania:Nudibranch effect > 0)",
    "P(Lafoea:Nudibranch effect > 0)",
    "P(Sertulariidae_Bryozoa:Nudibranch effect > 0)",
    "P(Other Hydroids:Nudibranch effect < 0)",
    "P(Sertulariidae:Nudibranch effect < 0)",
    "P(Hydrallmania:Nudibranch effect < 0)",
    "P(Lafoea:Nudibranch effect < 0)",
    "P(Sertulariidae_Bryozoa:Nudibranch effect < 0)"
  ),
  PosteriorProbability = c(
    pp_Other_Hydroids_nudie_positive,
    pp_Sertulariidae_nudie_positive,
    pp_Hydrallmania_nudie_positive,
    pp_Lafoea_nudie_positive,
    pp_Sertulariidae_Bryozoa_nudie_positive,
    pp_Other_Hydroids_nudie_negative,
    pp_Sertulariidae_nudie_negative,
    pp_Hydrallmania_nudie_negative,
    pp_Lafoea_nudie_negative,
    pp_Sertulariidae_Bryozoa_nudie_negative
  )
)

# Save table
write.table(pp_summary_m1v2_microhabitat_nudibranch, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/tables/table_postprob_m1v2_microhabitat_nudibranch.csv", sep = ",", row.names = TRUE, col.names = TRUE)

In [ ]:
# Generate HTML table
html_table <- minimal_html_table(pp_summary_m1v2_microhabitat_nudibranch, caption = "Posterior probabilities for Microhabitat*Nudibranch Interaction Effect")

# Display it as HTML
IRdisplay::display_html(html_table)


---

### 8. Summarize results

Sertulariidae with Bryozoans and Aeolid Nudibranch occurrence have a significantly positive effect on the log-odds of P. cristatus occurrence, with 85% and 95% confidence intervals supporting this relationship. In models that do not account for nudibranch-microhabitat associations, the Lafoea habitat appears potentially significant at the 85% confidence level. However, when nudibranch-microhabitat associations are included in the analysis, the effect of Lafoea microhabitat is no longer significant, while the effects of Sertulariidae with Bryozoans and Nudibranch occurrence remain significant, suggesting that the apparent significance of Lafoea in the initial model was driven by its association with shared microhabitats involving nudibranchs.

#### **Make final figures**

Generate tidy figures for the Results section of our report/paper. We will limit the results to showing only Microhabitat categories and add the intercept category at 0 for comparison.


In [ ]:
##### FINAL PLOT FOR WNAN PAPER

# Define the predictor list
predictors_m1v2 <- list(
  list(
    name = "Microhabitat",
    model = "m1",
    baseline = tibble(parameter = "Red_Algae", mean = 0),
    order = c("Red_Algae", "b_MicrohabitatOther_Hydroids", "b_MicrohabitatSertulariidae", "b_MicrohabitatHydrallmania", "b_MicrohabitatLafoea", "b_MicrohabitatSertulariidae_Bryozoa"),
    labels = c(
      "Red_Algae" = "Red algae",
      "b_MicrohabitatOther_Hydroids" = "Other hydroids",
      "b_MicrohabitatSertulariidae" = "Sertulariidae",
      "b_MicrohabitatHydrallmania" = "Hydrallmania",
      "b_MicrohabitatLafoea" = "Lafoea",
      "b_MicrohabitatSertulariidae_Bryozoa" = "Bryozoa"
    ),
    regex_pars = c(
      "^b_MicrohabitatOther_Hydroids$",
      "^b_MicrohabitatSertulariidae$",
      "^b_MicrohabitatHydrallmania$",
      "^b_MicrohabitatLafoea$",
      "^b_MicrohabitatSertulariidae_Bryozoa$")
  ),
  list(
    name = "Nudibranch",
    model = "m1",
    baseline = tibble(parameter = "Aeolid Absence", mean = 0),
    order = c("Aeolid Absence", "b_Nudibranch1"),
    labels = c(
      "Aeolid Absence" = "Aeolid absent",
      "b_Nudibranch1" = "Aeolid present"
    ),
    regex_pars = c("^b_Nudibranch1$")
  )
)

# Posterior samples
posterior_samples_m1v2 <- posterior_samples_m1v2  # This is the *single* posterior data.frame


# Helper: extract posterior + baseline
extract_effects <- function(posterior_df, predictor_cfg, predictor_name) {
  matched_cols <- posterior_df %>%
    dplyr::select(matches(predictor_cfg$regex)) %>%
    colnames()

  if (length(matched_cols) == 0) {
    stop(paste("No columns matched regex:", predictor_cfg$regex, "for predictor:", predictor_cfg$name))
  }

  draws <- posterior_df %>%
    dplyr::select(all_of(matched_cols)) %>%
    pivot_longer(cols = everything(), names_to = "parameter", values_to = "value") %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name
    )

  baseline <- predictor_cfg$baseline %>%
    mutate(
      label = predictor_cfg$labels[parameter],
      predictor = predictor_name,
      group = predictor_cfg$name,
      value = mean
    )

  bind_rows(draws, baseline)
}


# Extract all predictor draws
plot_data_m1v2 <- map_dfr(
  predictors_m1v2,
  ~ extract_effects(posterior_samples_m1v2, .x, .x$name)
)

# Factor label order
label_order_m1v2 <- c(
  "Red algae", "Other hydroids", "Sertulariidae", "Hydrallmania", "Lafoea", "Bryozoa",
  "Aeolid absent", "Aeolid present"
)
plot_data_m1v2$label <- factor(plot_data_m1v2$label, levels = label_order_m1v2)

# Final plot (single panel!)
final_plot_posteriors_m1v2 <- ggplot(plot_data_m1v2, aes(x = value, y = label, fill = group, color = group)) +
  stat_halfeye(
    .width = c(0.85, 0.95),
    slab_alpha = 0.4,
    interval_size_range = c(0.75, 0),
    normalize = "groups",
    slab_linewidth = 0.6,
    point_size = 1.25
  ) +
  geom_point(
    data = filter(plot_data_m1v2, !is.na(mean)),
    aes(x = mean),  # ✅ Match point color to group
    inherit.aes = TRUE,
    size = 0.5,
    shape = 21
  ) +
  geom_vline(xintercept = 0, size = 0.6, linetype = 3, color = "red") +
  scale_fill_manual(
    values = c(
      "Microhabitat" = "forestgreen",
      "Nudibranch" = "red"
    )
  ) +
  scale_color_manual(  # ✅ Add corresponding point colors
    values = c(
      "Microhabitat" = "forestgreen",
      "Nudibranch" = "red"
    )
  ) +
  labs(
    x = "Effect size (logit scale)",
    y = NULL
  ) +
  theme_bw(base_size = 8) +
  theme(
    strip.text = element_text(face = "bold", size = 10),
    axis.text.y = element_text(size = 8),
    panel.spacing = unit(0.5, "lines"),
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    legend.position = "none"
  )


# Save
ggsave("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m1v2.png",
       plot = final_plot_posteriors_m1v2, width = 4.5, height = 4.5, units = "in", dpi = 300)

In [ ]:
# Convert images to base64
final_plot_posteriors_m1v2 <- knitr::image_uri("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/images/final_plot_posteriors_m1v2.png")


# Create the HTML 
html_posteriors_final <- paste0("
  <style>
    .image-container {
      display: flex;
      justify-content: space-between;
      align-items: flex-start;
      flex-wrap: wrap;
      gap: 20px;
      max-width: 100%;
    }

    .image-container img {
      flex: 1 1 48%;
      max-width: 48%;
      height: auto;
      border: 1px solid #ccc;
    }

    @media screen and (max-width: 800px) {
      .image-container img {
        max-width: 100%;
        flex: 1 1 100%;
      }
    }
  </style>

  <div class='image-container'>
    <img src='", final_plot_posteriors_m1v2, "' alt='Final Posterior Plot'>
  </div>
")

# Display the HTML
IRdisplay::display_html(html_posteriors_final)